In [ ]:
# default_exp play03_dng
# pip3 install --user opencv-python opencv-contrib-python rawpy tqdm


In [ ]:
#export
import time
import pathlib
import numpy as np
import cv2 as cv
import pandas as pd
import xarray as xr
import rawpy
import cv2.aruco
import tqdm


In [ ]:
#export
_code_git_version="f86745e09534ee036aa93c2afccf7f6f5ded2293"
_code_repository="https://github.com/plops/cl-py-generator/tree/master//example/76_opencv_cuda/source/03_load_dng.ipynb"
_code_generation_time="18:17:34 of Wednesday, 2024-04-24 (GMT+1)"
start_time=time.time()
debug=True


In [ ]:
#export
df_status=pd.DataFrame([dict(name="numpy", version=np.__version__), dict(name="cv2", version=cv.__version__), dict(name="pandas", version=pd.__version__), dict(name="xarray", version=xr.__version__), dict(name="rawpy", version=rawpy.__version__)])
print(df_status)


In [ ]:
#export
# https://answers.opencv.org/question/98447/camera-calibration-using-charuco-and-python/
aruco_dict=cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_1000)
squares_x=49
squares_y=28
square_length=2
marker_length=1
board=cv.aruco.CharucoBoard_create(squares_x, squares_y, square_length, marker_length, aruco_dict)
out_size=(1960,1120,)
board_img=board.draw(out_size)
steps_x=5
steps_y=5


In [ ]:
#export
xs_fn="calib/checkerboards.nc"
if ( pathlib.Path(xs_fn).exists() ):
    start=time.time()
    xs=xr.open_dataset(xs_fn)
    print("duration loading from netcdf {:4.2f}s".format(((time.time())-(start))))
else:
    start=time.time()
    fns=list(pathlib.Path("/home/martin/stage/cl-py-generator/example/76_opencv_cuda/source/calib/").glob("APC*.dng"))
    res=[]
    for fn in tqdm.tqdm(fns):
        with rawpy.imread(str(fn)) as raw:
            res.append(raw.postprocess())
    data=np.stack(res, 0)
    xs=xr.Dataset(dict(cb=xr.DataArray(data=data, dims=["frame", "h", "w", "ch"], coords=dict(frame=np.arange(data.shape[0]), h=np.arange(data.shape[1]), w=np.arange(data.shape[2]), ch=np.arange(data.shape[3])))))
    xs.to_netcdf(xs_fn)
    print("duration loading from dng and saving netcdf {:4.2f}s".format(((time.time())-(start))))


In [ ]:
#export
w="cb"
cv.namedWindow(w, cv.WINDOW_NORMAL)
cv.resizeWindow(w, 800, 600)
decimator=0
all_corners=[]
all_ids=[]
for frame in range(len(xs.frame)):
    rgb=xs.cb[frame,...].values
    gray=cv.cvtColor(rgb, cv.COLOR_BGR2GRAY)
    markers=cv.aruco.detectMarkers(gray, aruco_dict)
    if ( ((0)<(len(markers[0]))) ):
        corners, ids, num=cv.aruco.interpolateCornersCharuco(markers[0], markers[1], gray, board)
        if ( (((corners is not None)) and ((ids is not None)) and (((3)<(corners))) and (((0)==(((decimator)%(3)))))) ):
            all_corners.append(corners)
            all_ids.append(ids)
        cv.aruco.drawDetectedMarkers(gray, markers[0], markers[1])
    cv.imshow(w, gray)
    cv.setWindowTitle(w, "frame {}".format(frame))
    cv.waitKey(20)
    decimator += 1
try:
    cal=cv.aruco.calibrateCameraCharuco(all_corners, all_ids, board, gray.shape, None, None)
except Exception as e:
    print(e)
    pass
cv.destroyAllWindows()
